# Using A/B Test to Determine the Best Mailer

## Project Overview

In this project we apply Chi-Square Test For Independence (a Hypothesis Test) to assess the performance of two types of mailers that were sent out to promote a new service! 

## Business Problem

Earlier in the year, our client, a grocery retailer, ran a campaign to promote their new "Delivery Club" - an initiative that costs a customer \\$100 per year for membership, but offers free grocery deliveries rather than the normal cost of \\$10 per delivery.

For the campaign promoting the club, customers were put randomly into three groups - the first group received a low quality, low cost mailer, the second group received a high quality, high cost mailer, and the third group were a control group, receiving no mailer at all.

The client knows that customers who were contacted, signed up for the Delivery Club at a far higher rate than the control group, but now want to understand if there is a significant difference in signup rate between the cheap mailer and the expensive mailer.  This will allow them to make more informed decisions in the future, with the overall aim of optimising campaign ROI!
***

## Data Understanding

In [7]:
import pandas as pd
from scipy.stats import chi2_contingency, chi2

In [8]:
campaign_data = pd.read_excel("data/grocery_database.xlsx", sheet_name = "campaign_data")

In [9]:
campaign_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 870 entries, 0 to 869
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   customer_id    870 non-null    int64         
 1   campaign_name  870 non-null    object        
 2   campaign_date  870 non-null    datetime64[ns]
 3   mailer_type    870 non-null    object        
 4   signup_flag    870 non-null    int64         
dtypes: datetime64[ns](1), int64(2), object(2)
memory usage: 34.1+ KB


In [10]:
campaign_data.head()

,customer_id,campaign_name,campaign_date,mailer_type,signup_flag
0,74,delivery_club,2020-07-01,Mailer1,1
1,524,delivery_club,2020-07-01,Mailer1,1
2,607,delivery_club,2020-07-01,Mailer2,1
3,343,delivery_club,2020-07-01,Mailer1,0
4,322,delivery_club,2020-07-01,Mailer2,1


In [11]:
campaign_data['mailer_type'].unique()

array(['Mailer1', 'Mailer2', 'Control'], dtype=object)

There are 711 mail-out in our campaign dataset with three mailer types 
- Mailer 1 being the low-quality-low-cost mailer;
- Mailer 2 being the high-quality-high-cost mailer;
- Control Group: receiving no mailers

The `signup_flag` (either 0 or 1) tells us whether a customer sign the membership.

## Data Preparation 

Since we only want to test whether a high-quality mailer type will attract more sign-up, we will just need the two categories of data - Mailer 1 and Mailer 2. We will exclude the `control Group` for this task.

In [15]:
# remove the data where the mail_type = 'control'
campaign_data = campaign_data.loc[campaign_data['mailer_type'] != 'Control']

#### 1) State hypothese & Set acceptance criteria

In [17]:
null_hypothesis = "There is no relationship between mailer type and signup rate. They are independent."
alternate_hypothesis = "There is a relationship between mailer type and signup rate. They are not independent."

In [18]:
acceptance_criteria = 0.05

#### 2) Summarize the data to get observed value

In [20]:
# aggregate our data to get the observed frequencies
pd.crosstab(campaign_data['mailer_type'], campaign_data['signup_flag'])

signup_flag,0,1
mailer_type,,
Mailer1,252,123
Mailer2,209,127


In [21]:
observed_values = pd.crosstab(campaign_data['mailer_type'], campaign_data['signup_flag']).values

In [22]:
mailer1_signup_rate = 123/(252+123)
mailer2_signup_rate = 127/(209+127)
print(f'mailer1_signup_rate: {mailer1_signup_rate} \nmailer2_signup_rate: {mailer2_signup_rate}')

mailer1_signup_rate: 0.328 
mailer2_signup_rate: 0.37797619047619047


From our observation, the mailer 2 resulted in a slightly higher sign-up rate. However, we need to test whether the higher sigh-up rate is statistically significant or it is just by chance to be higher.

#### 3) Run the chi square test

In [25]:
# calculate the chi squre statistics and the p_value
chi2_statistic , p_value, dof, expected_values = chi2_contingency(observed_values,correction = False)
print(f'chi2_statistic: {chi2_statistic}, p_value: {p_value}')

chi2_statistic: 1.9414468614812481, p_value: 0.16351152223398197


In [26]:
# Find the critical value for our test
critical_value = chi2.ppf(1 - acceptance_criteria, dof)
print(critical_value)

3.841458820694124


## Analysing The Results

Just from the results above we can see that our p-value of **0.16** is *greater* than our Acceptance Criteria of 0.05. Hence we will _retain_ the Null Hypothesis and conclude that there is no significant difference between the signup rates of Mailer 1 and Mailer 2.

We can make the same conclusion based upon our Chi-Square statistic of **1.94** being _lower_ than our Critical Value of **3.84**.
***

Let's create code to automatically print and interpret the results.

In [30]:
# print the results based upon Chi Square Statistic
if chi2_statistic >= critical_value:
    print(f"As our chi-squre statistic of {chi2_statistic} is higher than our critical value of {critical_value} - we reject the null hypothesis, and conclude that: {alternate_hypothesis}.")
else:   
    print(f"As our chi-squre statistic of {chi2_statistic} is lower than our critical value of {critical_value} - we retain the null hypothesis, and conclude that: {null_hypothesis}")

As our chi-squre statistic of 1.9414468614812481 is lower than our critical value of 3.841458820694124 - we retain the null hypothesis, and conclude that: There is no relationship between mailer type and signup rate. They are independent.


In [31]:
# Print the results based upon p_value
if p_value <= acceptance_criteria:
    print(f"As our p_value of {p_value} is lower than our critical value of {acceptance_criteria} - we reject the null hypothesis, and conclude that: {alternate_hypothesis}.")
else:
    print(f"As our p_value of {p_value} is higher than our critical value of {acceptance_criteria} - we retain the null hypothesis, and conclude that: {null_hypothesis}")

As our p_value of 0.16351152223398197 is higher than our critical value of 0.05 - we retain the null hypothesis, and conclude that: There is no relationship between mailer type and signup rate. They are independent.


## Business Impact and Application

Without running this Hypothesis Test, the client may have concluded that they should always look to go with higher cost mailers - and from what we've seen in this test, that may not be a great decision.  It would result in them spending more, but not *necessarily* gaining any extra membership sign-up and revenue as a result.

Our results here also do not say that there *definitely isn't a difference between the two mailers* - we are only advising that we should not make any rigid conclusions *at this point*.  

Running more A/B Tests like this, gathering more data, and then re-running this test may provide us, and the client more insight!

***